# Final Project 

In [3]:
from math import inf

You will use the following class to represent vertices in your graph. A vertex has an identifier, and it can also have a set of attributes. An identifier is almost like an attribute, except that it needs to be provided to create a vertex.

In [53]:
class Vertex:
    
    def __init__(self,id):
        self.attributes = {}
        self.attributes['id'] = id
        
    def __str__(self):
        return str(self.attributes)
        
    def new_copy(self):
        return Vertex(self.attributes['id'])
        
    def set(self,key,value):
        self.attributes[key] = value
        
    def get(self,key):
        return self.attributes[key]

In [54]:
class Graph:
    
    def __init__(self):
        self.vertices = {}
        self.id_to_v = {}
    
    def __str__(self):
        s = ''
        for v in self.vertices:
            s += str(v)
            s += '\n\n'
        return s
    
    def add_vertex(self,v):
        self.vertices[v] = []
        self.id_to_v[v.get('id')] = v
    
    def size_vertices(self):
        return len(self.vertices)
    
    def add_edge(self,v1,v2):
        pass
    
    def adjacent(self,v):
        return self.vertices[v]
    
    def BFS(self,start):
        assert(isinstance(start,Vertex))
        result = []
        
        start.set('color','gray')
        start.set('d',0)
        start.set('parent',None)
        
        for v in self.vertices:
            if v == start:
                continue
            
            v.set('color','white')
            v.set('d',inf)
            v.set('parent',None)
        
        queue = []
        queue.append(start)
        
        while len(queue) != 0:
            
            u = queue.pop(0)
            
            for v in self.adjacent(u):
                
                if v.get('color') == 'white':
                    
                    v.set('color','gray')
                    v.set('d',u.get('d') + 1)
                    v.set('parent',u)
                    
                    queue.append(v)
                    
            u.set('color','black')
            result.append(u.get('id'))

        return result
    
    def DFS_Visit(self,u,time):
        
        time += 1
        u.set('d',time)
        u.set('color','gray')
    
        for v in self.adjacent(u):
            
            if v.get('color') == 'white':
                v.set('parent',u)
                time = self.DFS_Visit(v,time)
    
        u.set('color','black')
        time += 1
        u.set('f',time)
    
        return time
    
    def DFS(self):
        
        for v in self.vertices:
            
            v.set('color','white')
            v.set('parent',None)
            
        time = 0
        
        for v in self.vertices:
            
            if v.get('color') == 'white':
                time = self.DFS_Visit(v,time)         

Your library will provide undirected graphs. Note that you may not change the name of the class, but you can change its inheritance, and you will need to add methods. 

In [55]:
class UndirectedGraph(Graph):
        
    def add_edge(self,v1,v2):
        self.vertices[v1].append(v2)
        self.vertices[v2].append(v1)

Likewise, your library will provide directed graphs.

In [56]:
class DirectedGraph(Graph):
        
    def add_edge(self,v1,v2):
        self.vertices[v1].append(v2)
        
    def transpose(self):
        GT = DirectedGraph()
        
        for v in self.vertices:
            GT.add_vertex(v)
            
        for v in self.vertices:
            for u in self.adjacent(v):
                GT.add_edge(u,v)
        
        return GT
        
    def acyclic(self):
        found_BE = False
        
        for u in self.vertices:
            for v in self.adjacent(u):
                b1 = v.get('d') <= u.get('d')
                b2 = u.get('d') < u.get('f')
                b3 = u.get('f') <= v.get('f')
                if b1 and b2 and b3: 
                    found_BE = True
                    
        return not found_BE
        
    def topological_sort(self):
        self.DFS()
        if self.acyclic():
            result = sorted(self.vertices,key=lambda v: v.get('f'),reverse=True)
            return list(map(lambda v: v.get('id'),result))
        else:
            return None

Both kinds of graphs should have an API that allows us to create new graphs. First, there should be a method 'add_vertex' that takes a vertex as an input and adds it to the graph. Second, there should be a method 'add_edge' that takes two vertices and adds an edge between them in the graph. For example, your implementation should work with the following codes.

In [57]:
def create_graph_1():
    G = UndirectedGraph()

    for i in ['r','s','t','u','v','w','x','y']:
        G.add_vertex(Vertex(i))
    
    for (v1,v2) in [('v','r'),('r','s'),('s','w'),
              ('w','t'),('w','x'),('t','x'),
              ('t','u'),('x','u'),('x','y'),('u','y')]:
    
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
    
    return G

In [58]:
G1 = create_graph_1()

In [59]:
def create_graph_2():
    G = DirectedGraph()
    
    for i in ['u','v','w','x','y','z']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2) in [('u','x'),('u','v'),('x','v'),('v','y'),
                    ('y','x'),('w','y'),('w','z'),('z','z')]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
        
    return G

In [60]:
G2 = create_graph_2()

In [61]:
def create_graph_3():
    G = DirectedGraph()
    
    for i in ['u','v','w','x','y','z']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2) in [('u','x'),('u','v'),('v','y'),
                    ('y','x'),('w','y'),('w','z')]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
        
    return G    

In [62]:
G3 = create_graph_3()

Please note two important things. First, the methods add_vertex and add_edge do not return a new graph, instead they update the graph in-place. Second, in the graph implementation, you need to work with vertex objects, and not their identifier. It means that to interface with the graph, the graph needs to provide a dictionnary 'id_to_v' that keeps track of which identifier corresponds to which vertex.

**Problem 1: (20 points)** Implement Breads-First Search (BFS) for both kinds of graphs. The BFS method takes a vertex as an argument and should return a list of identifiers. Note that there can be more than one right answer. *The following call should work.*

In [63]:
G1.BFS(G1.id_to_v['s'])

['s', 'r', 'w', 'v', 't', 'x', 'u', 'y']

**Problem 2: (20 points)** Implement Depth-First Search (BFS) for both kinds of graphs. The DFS method takes no arguments and does not return anything. Instead, the DFS method should timestamp the vertices. *The following call should work.*

In [64]:
G2.DFS()

**Problem 3: (20 points)** Implement a method to test if a directed graph is acyclic. The methods takes no argument but returns a boolean. *The following call should work.*

In [65]:
G3.DFS()
G3.acyclic()

True

**Problem 4: (20 points)** Implement a method that computes a topological sort for a directed graph. The method takes no arguments and returns a list of identifiers. *The following call should work.*

In [66]:
G3.topological_sort()

['w', 'z', 'u', 'v', 'y', 'x']

#### Kosaraju Algorithm

In [67]:
class KosarajuGraph(DirectedGraph): # customized graph to apply Kosaraju Algorithm
    def __init__(self):
        super().__init__()
        self.stack = []
        
    def transpose(self):
        # self : original Graph, GT : transposed Graph 
        GT = KosarajuGraph()
        
        for v in self.vertices:
            # v : original vertex, gv : new vertex with identical value to original vertex (for GT)             
            gv = Vertex(v.get('id'))
            
            GT.add_vertex(gv)
            
        for gv in GT.vertices:
            v = self.id_to_v[gv.get('id')]
            for u in self.adjacent(v):
                gu = GT.id_to_v[u.get('id')]
                GT.add_edge(gu,gv)
        
        return GT
        
        
    def get_stack(self):
        return self.stack
    
    def init_vertex(self):
        for v in self.vertices:            
            v.set('color', 'white')
            v.set('parent', None)
    
    def DFS_Visit(self,u,time):
        
        time += 1
        u.set('d',time)
        u.set('color','gray')
    
        for v in self.adjacent(u):
            
            if v.get('color') == 'white':
                v.set('parent',u)
                time = self.DFS_Visit(v,time)
    
        u.set('color','black')
        time += 1
        u.set('f',time)

        self.stack.append(u)
        return time
    
    def T_Visit(self, u):
        u.set('color','gray')
        self.stack.append(u)
        for v in self.adjacent(u):            
            if v.get('color') == 'white':
                v.set('parent',u)
                self.T_Visit(v)
    
        u.set('color','black')

class WeightedUndirectedGraph(UndirectedGraph):
    def __init__(self):
        super().__init__()
        self.weights = {}    
    
    def add_edge(self, v1, v2, weight):
        super().add_edge(v1, v2)
        self.weights[(v1,v2)] = weight
        self.weights[(v2,v1)] = weight
        
    
class WeightedDirectedGraph(DirectedGraph):
    class _MinHeap:
        def __init__(self):
            self.heap_list = [0]
            self.id = [0]
            self.current_size = 0

        def swim(self, i):
            while i // 2 > 0:
                if self.heap_list[i] < self.heap_list[i // 2]:
                    self.heap_list[i], self.heap_list[i // 2] = self.heap_list[i // 2], self.heap_list[i]
                    self.id[i], self.id[i // 2] = self.id[i // 2], self.id[i]
                i = i // 2

        def insert(self, k, id):
            self.heap_list.append(k)
            self.id.append(id)
            self.current_size += 1
            self.swim(self.current_size)

        def min_child(self, i):
            if (i * 2)+1 > self.current_size:
                return i * 2
            else:
                if self.heap_list[i*2] < self.heap_list[(i*2)+1]:
                    return i * 2
                else:
                    return (i * 2) + 1        

        def sink(self, i):
            while (i * 2) <= self.current_size:
                mc = self.min_child(i)
                if self.heap_list[i] > self.heap_list[mc]:
                    self.heap_list[i], self.heap_list[mc] = self.heap_list[mc], self.heap_list[i]
                    self.id[i], self.id[mc] = self.id[mc], self.id[i]
                i = mc

        def delete_min(self):
            if len(self.heap_list) == 1:
                return None

            root = self.heap_list[1]
            root_id = self.id[1]
            self.heap_list[1] = self.heap_list[self.current_size]
            self.id[1] = self.id[self.current_size]
            *self.heap_list, _ = self.heap_list
            *self.id, _ = self.id
            self.current_size -= 1
            self.sink(1)
            return (root, root_id)    

    def __init__(self):
        super().__init__()
        self.weights = {}
        self.heap = self._MinHeap()
        
    def add_edge(self, v1, v2, weight):
        super().add_edge(v1, v2)
        self.weights[(v1,v2)] = weight


In [68]:
class SCC:
    def __init__(self):
        self.group = []

    def append(self, stack, G):
        subGraph = KosarajuGraph()
        
        for v in stack:
            sv = Vertex(v.get('id'))
            subGraph.add_vertex(sv)

        for sv in subGraph.vertices:
            v = G.id_to_v[sv.get('id')]
            
            for u in G.adjacent(v):
                if u.get('id') in subGraph.id_to_v:
                    su = subGraph.id_to_v[u.get('id')]
                    subGraph.add_edge(sv,su)
                    
        self.group.append(subGraph)
    
    def __getitem__(self, i):
        return self.group[i]
    
    def __repr__(self):
        result = "<SCC.kosaraju.Algrothms>\n"
        result += self.__str__()
        
        return result

    def __len__(self):
        return len(self.group)
    
    def __str__(self):
        result = ""
        
        result += "# of SCC : {0}\n".format(len(self.group))
        result += "---------\n"
        result += "SCC \n"
        result += "---------\n"
        
        for g in self.group:
            for s in g.vertices:
                ts = g.vertices[s]
                if len(ts) == 0:
                    result += "{}\n".format(s.get('id'))
                for t in ts:
                    result += "{} -> {}\n".format(s.get('id'), t.get('id'))
                    
            result += "---------\n"
            
        return result
    
class MinimumSpanningTree:
    def __init__(self):
        self.edges = []
        self.weights = []

    def append(self, v1, v2, w):
        self.edges.append((v1, v2))
        self.weights.append(w)
        
    def __repr__(self):
        result = "<MST.kruskal.Algrothms>\n"
        result += self.__str__()
        
        return result

    def __len__(self):
        return len(self.edges)
    
    def __str__(self):
        result = ""
        
        result += "Total cost of MST : {0}\n".format(sum(self.weights))
        result += "---------\n"
        result += "MST \n"
        result += "---------\n"

        for w, (s, t) in zip(self.weights, self.edges):
            result += "{} --({})-- {}\n".format(s, w, t)                    
            
        return result


In [69]:
class Algorithms:
    class _Sets:
        def __init__(self, vertices):
            self.sets = [set([v.get('id')]) for v in vertices]
        
        def find_set(self, x):
            for s in self.sets:
                if x in s:
                    return s

        def union_set(self, s1, s2):
            memory = 0
            for i, s in enumerate(self.sets):
                if s == s1:
                    memory = i
            self.sets.pop(memory)
            memory = 0
            for i, s in enumerate(self.sets):
                if s == s2:
                    memory = i
            self.sets.pop(memory)
            self.sets.append(s1 | s2)
    
    def __init__(self):
        pass
    
    def kosaraju(self, G):
        assert isinstance(G, KosarajuGraph), "Given graph is not a directed graph"
        scc = SCC()
        
        T = G.transpose()
        G.DFS() # Store each vertex from DFS-process inside the stack 
        stack = G.get_stack()
        
        T.init_vertex()
        
        while len(stack) != 0:
            v = stack.pop()
            u = T.id_to_v[v.get('id')]
            
            if u.get('color') == 'white':
                T.T_Visit(u)
                scc.append(T.get_stack(), G)
                T.stack = []
                
        return scc
    

    def kruskal(self, G):
        assert isinstance(G, WeightedUndirectedGraph), "Given graph is not a weighted undirected graph"
        tree = MinimumSpanningTree()
        sets = self._Sets(G.vertices)
        
        
        sorted_edges = sorted(G.weights.items(), key=lambda x: x[1])
        for ((v, u),w) in sorted_edges:
            s1 = sets.find_set(v.get('id'))
            s2 = sets.find_set(u.get('id'))
            if s1 != s2:
                tree.append(u.get('id'), v.get('id'), w)
                sets.union_set(s1, s2)

        return tree
    
    def dijkstra(self, G, start):
        assert isinstance(G, WeightedDirectedGraph), "Given graph is not a weighted directed graph"
        distances = {node.get('id') : float('infinity') for node in G.vertices}

        distances[start] = 0 

        
        G.heap.insert(distances[start], start)

        while G.heap.current_size > 0:
            current_distance, current_destination = G.heap.delete_min()

            if distances[current_destination] < current_distance:
                continue

            node = G.id_to_v[current_destination]
            adj_v = G.vertices[node] ## adj_v = [vertex1, vertex2, ...], node = vertex0
            weights = [G.weights[(node, v)] for v in adj_v]

            for new_dest_v, new_distance in zip(adj_v, weights):
                new_destination = new_dest_v.get('id')
                distance = current_distance + new_distance

                if distance < distances[new_destination]:
                    distances[new_destination] = distance
                    G.heap.insert(distance, new_destination)

        return ("Dijkstra Result", distances)

#### Test Kosaraju

In [70]:
def create_graph_4():
    G = KosarajuGraph()
    
    for i in ['a','b','c','d','e','f','g','h','i','j','k','l','m']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2) in [('a','b'),('b','c'),('c','d'), ('d','a'), ('b', 'e'),
                    ('d','f'),('c','g'),('g','h'), ('h', 'i'),
                   ('i','j'),('j','g'),('h','k'), ('k', 'h'), ('k', 'l'), ('l', 'm'), ('m', 'l')]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
        
    return G    

In [71]:
solver = Algorithms()

In [72]:
G4 = create_graph_4()
scc = solver.kosaraju(G4)

In [73]:
scc

<SCC.kosaraju.Algrothms>
# of SCC : 5
---------
SCC 
---------
a -> b
d -> a
c -> d
b -> c
---------
e
---------
g -> h
j -> g
i -> j
h -> i
h -> k
k -> h
---------
l -> m
m -> l
---------
f
---------

#### Test Kruskal

In [74]:
def create_graph_5():
    G = WeightedUndirectedGraph()
    
    for i in ['a','b','c','d','e','f','g']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2,w) in [('a', 'b', 29), ('b','c',16), ('b','g',15), ('c','d',12), ('g', 'd', 18), ('d', 'e', 22), ('e', 'g', 25), ('e','f',27), ('f','a',10)]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2], w)
        
    return G    

In [75]:
G5 = create_graph_5()
mst = solver.kruskal(G5)

In [76]:
mst

<MST.kruskal.Algrothms>
Total cost of MST : 102
---------
MST 
---------
a --(10)-- f
d --(12)-- c
g --(15)-- b
c --(16)-- b
e --(22)-- d
f --(27)-- e

#### Test Dijkstra

In [77]:
def create_graph_6():
    G = WeightedDirectedGraph()
    
    for i in ['a','b','c','d','e','f']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2,w) in [('a','b', 1), ('a','c', 4), ('a','d', 9), ('c','b', 4), ('c', 'd', 2), ('d', 'e', 10), 
                      ('d', 'f', 9), ('e','f', 4), ('f','a', 8)]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2], w)
        
    return G    

In [78]:
G6 = create_graph_6()
result = solver.dijkstra(G6, 'a')

In [79]:
result

('Dijkstra Result', {'a': 0, 'b': 1, 'c': 4, 'd': 6, 'e': 16, 'f': 15})